# Ejemplo 3:  Recorrido de Aves

<img src="buteo_albigula.jpg">

Un biólogo está estudiando el patrón de migración del ave *Buteo albigula* en Suramérica. Para esto accedió a la siguiente base de datos https://www.movebank.org  (Study-Marc Bechard Hawks) y obtuvo un archivo de puntos con la localización de un individuo para diferentes días entre enero y octubre de 2011.

Su primer objetivo consiste en visualizar el recorrido del ave a lo largo del tiempo y su segundo objetivo consiste en analizar el área donde el individuo se detiene al final del recorrido a partir de imágenes de Landsat 8

In [2]:
from arcgis import GIS,features
from arcgis.raster.functions import *   
import time 
from IPython import display

In [1]:
gis = GIS(username = 'rmartin_geek') 

NameError: name 'GIS' is not defined

In [ ]:
m = gis.map('Chile')
m

## Preparar datos y simbología
En esta sección vamos a:
1. Cargamos los datos 
2. Realizamos la conversión de `FeatureLayer` a `FeatureSet` dada la ventaja de este último objeto para graficar y realizar operaciones
3. Creamos las opciones para la simbología. 


### Cargar datos 


In [ ]:
aves= features.FeatureLayer('http://geoapps.esri.co:6080/arcgis/rest/services/No_Oficial/DatosButeoAlbigula/MapServer/0')

### Realizar conversión

In [ ]:
avesset = aves.query()
avesset

### Creación de simbología

In [ ]:
ave = {"type":"esriPMS",
       "url":"http://geoapps.esri.co/Recursos/webinarPython/avecafe.png",
       "contentType":"image/png","width":24,"height":8}

In [ ]:
linea = {"type": "esriSLS",
         "style": "esriSLSDash",
         "color": [121,85,61,255],
         "width": 2}

## Creación del recorrido 


In [ ]:
coords = [avesset.features[i].geometry for i in range(len(avesset.features))]
tiempo = 0.5
for i in range(len(avesset.features)):
    m.draw(coords[i], symbol=ave)
    m.center = coords[i]['y'], coords[i]['x']
        #m.clear_graphics()
    if i > 0:
            #for n in range(1,i):
        m.draw ({"paths" : [[[coords[i-1]['x'],coords[i-1]['y']], [coords[i]['x'],coords[i]['y']]]],
                    "spatialReference" : {"wkid" : 4326}}, symbol=linea)
    time.sleep(tiempo)


In [ ]:
m.clear_graphics()

## Análisis de imágenes raster de la zona de interés
En este punto es de interés para el científico hacer un análisis de coberturas en una zona que el desplegue en el mapa. para realizar esta labor es necesario buscar el servicio que contienen imágenes Landsat, generar una combinación de color que permita su fácil identificación y finalmente desplegar su visualización en el mapa.

In [ ]:
items = gis.content.search('landsat', 'imagery layer', outside_org=True)
for item in items:
    display.display(item)

In [ ]:
landsatservice = items[2]
landsat = landsatservice.layers[0]

Generamos una mejora a la imagen acorde a una combinación de color

In [ ]:
imagen = stretch(extract_band(landsat, [5,4,3]), 
                    stretch_type='StdDev',num_stddev=2.5, dra=True) 

Seleccionamos la imagen Landsat conforme a la extensión del mapa

In [ ]:
imagen.extent = m.extent
imagen

Acorde a una necesidad específica del biólogo requiere seleccionar una zona a su gusto por lo cual es necesario dibujar la zona que requiere y exportarla para almacenar en su WebGIS y ompartirla con sus compañeros científicos que comparten en la organización.

In [ ]:
def get_geom(m,g):
    global geom
    geom = g
    
m.on_draw_end(get_geom)


In [ ]:
m.draw('rectangle')

para realizar el corte de la zona que dibujó se utilizará la función `clip()`

In [ ]:
from arcgis.raster.functions import clip

In [ ]:
imagen_clip  = clip(landsat, geom)
m.add_layer(imagen_clip)

In [ ]:
imagen_clip.extent = geom['cache']['_extent']
imagen_clip


Para mejorar la visualización se realiza un ajuste visual usando la función `stretch()`

In [ ]:
strim= stretch(extract_band(imagen_clip, [5,4,3]), 
                    stretch_type='StdDev',num_stddev=2.5, dra=True)

In [ ]:
strim

In [ ]:
m.add_layer(strim)

In [ ]:
m.clear_graphics()

Finalmente guarda su resultado en su cuenta de ArcGIS Online y compartirlo en la organización

In [ ]:
publicada = strim.save(output_name='Raster ejmplo21', for_viz=True, gis = gis)
publicada

In [ ]:
publicada.share(org=True)